In [2]:
import requests
import lxml.html as lh
import pandas as pd

In [107]:
#importing csv and getting health expenditure as a % of OECD country's GDP
import csv
import os

headers = []
oecd_data = []

os.getcwd()

# only include health expenditure as a % of GDP of each country in 2018
def include_line(row):
     return 'Share of gross domestic product' in row and '2018' in row and 'All financing schemes' in row and 'Current expenditure on health (all functions)' in row
    

with open('OECD_data.csv') as file:
    f = csv.reader(file)
    headers = next(f)
    
    for row in f:
        if include_line(row):
            oecd_data.append(row)

# creating a dictionary
oecd_dict = {}
for header in headers:
    if header == 'Country' or header == 'Value':
        oecd_dict[header] = []

#adding data to each corresponding header
for elem in oecd_data:
    for header, data in oecd_dict.items():
        if header == 'Country':
            oecd_dict[header].append(elem[9])
        if header == 'Value':
            oecd_dict[header].append(elem[18])

oecd_df = pd.DataFrame(oecd_dict)
oecd_df

,Country,Value
0,Denmark,10.47
1,France,11.177
2,Greece,7.846
3,Italy,8.813
4,Norway,10.182
5,Portugal,9.099
6,Sweden,11.037
7,Turkey,4.172
8,United Kingdom,9.768
9,Australia,9.252


In [3]:
#Getting covid 19 cases in oecd countries from url and a 
covid_url = 'http://www.worldometers.info/coronavirus/?utm_campaign=homeAdvegas1?'
page = requests.get(covid_url)

In [83]:
#creating a correct html doc from the given string
doc = lh.fromstring(page.content)
xpath_rows = '//tr'
rows = doc.xpath(xpath_rows)



In [84]:
#Check the length of the first 12 rows
[len(T) for T in rows[:12]]

[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]

In [131]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
# Only get the following columns: country and total cases
for t in rows[0]:
    i+=1
    if i == 4:
        break
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))
    

1:"#"
2:"Country,Other"
3:"TotalCases"


In [139]:
def include_data(data):
    for i in range(3):
        col[i][1].append(data[i])
        
def meet_crit(data):
    # These are the countries that are named differently in the two datasets
    # For example in OECD = United States whereas in Worldometer = USA so we can't check if USA is in OECD dictionary
    countries = ['USA','UK', 'S. Korea', 'Slovakia', 'Czechia']
    return data in oecd_dict['Country'] or data in countries
    
# getting data from rows:
for j in range(1, len(rows)):
    row = rows[j];
    i = 0
    temp = []
    for t in row.iterchildren():
        data=t.text_content()
        if i == 3:
            break
        if i == 0 or i == 2:
            try: 
        #Convert any numerical value to integers
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        temp.append(data)
        #Increment i for the next column
        i+=1
    #only include covid data for OECD member countries and do not include duplicates
    if meet_crit(temp[1]):
        if temp[1] in col[1][1]:
            continue
        else:
            include_data(temp)



In [144]:
col[1][1]
#making sure covid data list and health expenditure list have the same number of countries
len(col[1][1]) == len(oecd_dict['Country'])


True

In [145]:
# get covid data for only OECD member countries
covid_dict = {}

for (header, data) in col:
    if header in oecd_dict['Country']:
        covid_dict[header]
dictionary = {header:data for (header, data) in col}
df = pd.DataFrame(dictionary)
df

,#,"Country,Other",TotalCases
0,1,USA,"2,345,371"
1,5,UK,"304,331"
2,6,Spain,"293,352"
3,8,Chile,"242,355"
4,9,Italy,"238,499"
5,11,Germany,"191,346"
6,12,Turkey,"187,685"
7,14,Mexico,"175,202"
8,15,France,"160,093"
9,18,Canada,"101,286"
